# Basic Import

In [1]:
import numpy as np
import meshio
import os
#import tensorflow as tf

In [2]:
import pyvista as pv

# Example of a simple mesh

In [3]:
mesh_path = 'shapes/shape_1_1_0.vtu'

### We can check the mesh with pyvista or paraview. Here we chose pyvista to stay in a notebook.

In [4]:
mesh_pv = pv.read(mesh_path)

In [ ]:
mesh_pv.plot(show_scalar_bar=True, show_axes=True, notebook = True,show_edges=True)

# Gradient

Let's compute the gradient of this field on this unstructured mesh. We first need to compute (and store to reuse them later) gradient matrix.

In [ ]:
mesh = meshio.read(mesh_path,"vtu")

In [ ]:
from meshgradient.matrix import build_CON_matrix, build_PCE_matrix, build_AGS_matrix

In [ ]:
%time test = build_CON_matrix(mesh)
print()
print('-'*10)
print()
print("Gradient Matrix as a Tensorflow Sparse Tensor: ")
print()
print(test)
print()
print('-'*10)
print()

Note that while we used these matrix to perform machine learning training within Tensorflow, you can replace these tensorflow Sparse tensors with any other sparse matrix of your choice.

Let's build and save our 3 gradient matrix: 

In [ ]:
import pickle
def build_and_save_gradient_matrix(mesh,gradient_folder,gradient_filename):
  tf_gradient_matrix = build_AGS_matrix(mesh)
  with open(os.path.join(gradient_folder,'AGS_' + gradient_filename), 'wb') as handle:
    pickle.dump(tf_gradient_matrix, handle, protocol=pickle.HIGHEST_PROTOCOL)

  tf_gradient_matrix = build_PCE_matrix(mesh)
  with open(os.path.join(gradient_folder,'PCE_' + gradient_filename), 'wb') as handle:
    pickle.dump(tf_gradient_matrix, handle, protocol=pickle.HIGHEST_PROTOCOL)

  tf_gradient_matrix = build_CON_matrix(mesh)
  with open(os.path.join(gradient_folder,'CON_' + gradient_filename), 'wb') as handle:
    pickle.dump(tf_gradient_matrix, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
build_and_save_gradient_matrix(mesh, 'shapes','example_gradient')

Let's use these matrix to compute our gradients. First we load the 3 tensor and then we use our built in functions to compute the gradient.

In [ ]:
def load_gradient_matrix(gradient_folder,gradient_filename):
  with open(os.path.join(gradient_folder,'AGS_' + gradient_filename), 'rb') as handle:
    tf_AGS_matrix = pickle.load(handle)

  with open(os.path.join(gradient_folder,'PCE_' + gradient_filename), 'rb') as handle:
    tf_PCE_matrix = pickle.load(handle)

  with open(os.path.join(gradient_folder,'CON_' + gradient_filename), 'rb') as handle:
    tf_CON_matrix = pickle.load(handle)
  return tf_AGS_matrix, tf_PCE_matrix, tf_CON_matrix

In [ ]:
gradient_matrix = load_gradient_matrix('shapes','example_gradient')

We will use 4 boundaries for the 4 sides of our cavity, and compute the gradient of the field "Erreur": 

In [ ]:
b1,b2,b3,b4 = mesh.point_data['AppartientEntree1'],mesh.point_data['AppartientEntree2'],mesh.point_data['AppartientEntree3'],mesh.point_data['AppartientEntree4']

In [ ]:
erreur_field = mesh.point_data['Erreur']

Let's import one of our function: 

In [ ]:
from meshgradient.gradient_fn import compute_gradient_per_points

In [ ]:
gradient_erreur = compute_gradient_per_points(gradient_matrix,erreur_field,b1,b2,b3,b4)

In [ ]:
gradient_erreur

Let's add this field to our mesh, and display it (or compare it in paraview with the built in gradient from paraview).

In [ ]:
mesh.point_data['Erreur'] = gradient_erreur.numpy()
meshio.write('shapes/shape_with_gradient.vtu',mesh)

In [ ]:
mesh_pv = pv.read('shapes/shape_with_gradient.vtu')

In [ ]:
mesh_pv.plot(show_axes=True, notebook = True,show_edges=False)